In [74]:
# IMPORTING MODULES
import glob
import importlib
import matplotlib.pyplot as plt
import numpy as np
import os
module_path = os.path.abspath(os.path.join('..', 'src'))
sys.path.append(module_path)
import pandas as pd
import scipy.signal as ss
import sys

import tools.data_reader as dr
import tools.display_tools as dt
import tools.preprocessing as preprocessing

from scipy.fft import fft, fftfreq, fftshift
from sklearn import svm
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

In [80]:
# SVM
importlib.reload(preprocessing)
importlib.reload(dr)
importlib.reload(dt)

import csv

metrics_folder = os.path.join(dr.Paths.DATA_DIR, "metrics")
metrics = ["bpm", "rmssd", "hf_rr", "lf_rr", "mean_SCL", "SCR_rate"]
phases = [
    dr.Phases.BASE_REST, dr.Phases.BASE_SPEECH,
    dr.Phases.BUG_RELAX, dr.Phases.BUG_ANTICIPATE, dr.Phases.BUG_EXPOSURE, dr.Phases.BUG_BREAK, dr.Phases.BUG_REFLECT,
    dr.Phases.SPEECH_RELAX, dr.Phases.SPEECH_ANTICIPATE, dr.Phases.SPEECH_EXPOSURE, dr.Phases.SPEECH_BREAK, dr.Phases.SPEECH_REFLECT

]

clf = svm.SVC(kernel='poly', degree=8)

for metric in metrics:
    print(f"SVM USING {metric}")
    ha = []
    la = []
    for phase in phases:
        file = os.path.join(metrics_folder, f"{metric}_{phase}_ha.csv")
        arr = pd.read_csv(file, header=None, index_col=[0]).to_numpy()
        avg_bpm = np.nanmean(arr[1:, 1:], axis=1)
        ha.append(avg_bpm)

        file = os.path.join(metrics_folder, f"{metric}_{phase}_la.csv")
        arr = pd.read_csv(file, header=None, index_col=[0]).to_numpy()
        avg_bpm = np.nanmean(arr[1:, 1:], axis=1)
        la.append(avg_bpm)

    ha_arr = np.asarray(ha).transpose()
    la_arr = np.asarray(la).transpose()
    print(np.any(np.isnan(ha_arr)))
    print(np.any(np.isnan(la_arr)))

    x = np.vstack([ha_arr, la_arr])
    y = np.asarray([1 for _ in range(ha_arr.shape[0])] + [0 for _ in range(ha_arr.shape[0])])

    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=16)

    clf.fit(x_train, y_train)

    preds = clf.predict(x_test)
    actual = y_test

    print(confusion_matrix(actual, preds))
    print(classification_report(actual, preds))

SVM USING bpm
False
False
[[2 0]
 [2 2]]
              precision    recall  f1-score   support

           0       0.50      1.00      0.67         2
           1       1.00      0.50      0.67         4

    accuracy                           0.67         6
   macro avg       0.75      0.75      0.67         6
weighted avg       0.83      0.67      0.67         6

SVM USING rmssd
True
True


C:\Users\zhoux\AppData\Local\Temp/ipykernel_27024/405361247.py:26: RuntimeWarning: Mean of empty slice
  avg_bpm = np.nanmean(arr[1:, 1:], axis=1)
C:\Users\zhoux\AppData\Local\Temp/ipykernel_27024/405361247.py:26: RuntimeWarning: Mean of empty slice
  avg_bpm = np.nanmean(arr[1:, 1:], axis=1)
C:\Users\zhoux\AppData\Local\Temp/ipykernel_27024/405361247.py:26: RuntimeWarning: Mean of empty slice
  avg_bpm = np.nanmean(arr[1:, 1:], axis=1)
C:\Users\zhoux\AppData\Local\Temp/ipykernel_27024/405361247.py:26: RuntimeWarning: Mean of empty slice
  avg_bpm = np.nanmean(arr[1:, 1:], axis=1)
C:\Users\zhoux\AppData\Local\Temp/ipykernel_27024/405361247.py:26: RuntimeWarning: Mean of empty slice
  avg_bpm = np.nanmean(arr[1:, 1:], axis=1)
C:\Users\zhoux\AppData\Local\Temp/ipykernel_27024/405361247.py:31: RuntimeWarning: Mean of empty slice
  avg_bpm = np.nanmean(arr[1:, 1:], axis=1)


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [78]:
# SVM -- ECG RMSSD
importlib.reload(preprocessing)
importlib.reload(dr)
importlib.reload(dt)

import csv

metrics_folder = os.path.join(dr.Paths.DATA_DIR, "metrics")
metric = "rmssd"
phases = [
    dr.Phases.BASE_REST, dr.Phases.BASE_SPEECH,
    dr.Phases.BUG_RELAX, dr.Phases.BUG_ANTICIPATE, dr.Phases.BUG_EXPOSURE, dr.Phases.BUG_BREAK, dr.Phases.BUG_REFLECT,
    dr.Phases.SPEECH_RELAX, dr.Phases.SPEECH_ANTICIPATE, dr.Phases.SPEECH_EXPOSURE, dr.Phases.SPEECH_BREAK, dr.Phases.SPEECH_REFLECT

]

ha = []
la = []

for phase in phases:
    file = os.path.join(metrics_folder, f"{metric}_{phase}_ha.csv")
    arr = pd.read_csv(file, header=None, index_col=[0]).to_numpy()
    avg_bpm = np.nanmean(arr[1:, 1:], axis=1)
    ha.append(avg_bpm)

    file = os.path.join(metrics_folder, f"{metric}_{phase}_la.csv")
    arr = pd.read_csv(file, header=None, index_col=[0]).to_numpy()
    avg_bpm = np.nanmean(arr[1:, 1:], axis=1)
    la.append(avg_bpm)

ha_arr = np.asarray(ha).transpose()
la_arr = np.asarray(la).transpose()

x = np.vstack([ha_arr, la_arr])
x = np.nan_to_num(x)
y = np.asarray([1 for _ in range(ha_arr.shape[0])] + [0 for _ in range(ha_arr.shape[0])])


clf = svm.SVC()
for i in [1, 12, 15, 16, 19, 22, 40, 41]:
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=i)

    clf.fit(x_train, y_train)

    preds = clf.predict(x_test)
    actual = y_test

    acc = accuracy_score(actual, preds)
    print(f"Acc: {acc}")

Acc: 0.6666666666666666
Acc: 0.3333333333333333
Acc: 0.8333333333333334
Acc: 0.6666666666666666
Acc: 0.5
Acc: 0.16666666666666666
Acc: 0.6666666666666666
Acc: 0.6666666666666666


C:\Users\zhoux\AppData\Local\Temp/ipykernel_27024/1598856489.py:23: RuntimeWarning: Mean of empty slice
  avg_bpm = np.nanmean(arr[1:, 1:], axis=1)
C:\Users\zhoux\AppData\Local\Temp/ipykernel_27024/1598856489.py:23: RuntimeWarning: Mean of empty slice
  avg_bpm = np.nanmean(arr[1:, 1:], axis=1)
C:\Users\zhoux\AppData\Local\Temp/ipykernel_27024/1598856489.py:23: RuntimeWarning: Mean of empty slice
  avg_bpm = np.nanmean(arr[1:, 1:], axis=1)
C:\Users\zhoux\AppData\Local\Temp/ipykernel_27024/1598856489.py:23: RuntimeWarning: Mean of empty slice
  avg_bpm = np.nanmean(arr[1:, 1:], axis=1)
C:\Users\zhoux\AppData\Local\Temp/ipykernel_27024/1598856489.py:23: RuntimeWarning: Mean of empty slice
  avg_bpm = np.nanmean(arr[1:, 1:], axis=1)
C:\Users\zhoux\AppData\Local\Temp/ipykernel_27024/1598856489.py:28: RuntimeWarning: Mean of empty slice
  avg_bpm = np.nanmean(arr[1:, 1:], axis=1)


In [69]:
# SVM -- ECG HF RR
importlib.reload(preprocessing)
importlib.reload(dr)
importlib.reload(dt)

import csv

metrics_folder = os.path.join(dr.Paths.DATA_DIR, "metrics")
metric = "hf_rr"
phases = [
    dr.Phases.BASE_REST, dr.Phases.BASE_SPEECH,
    dr.Phases.BUG_RELAX, dr.Phases.BUG_ANTICIPATE, dr.Phases.BUG_EXPOSURE, dr.Phases.BUG_BREAK, dr.Phases.BUG_REFLECT,
    dr.Phases.SPEECH_RELAX, dr.Phases.SPEECH_ANTICIPATE, dr.Phases.SPEECH_EXPOSURE, dr.Phases.SPEECH_BREAK, dr.Phases.SPEECH_REFLECT

]

ha = []
la = []

for phase in phases:
    file = os.path.join(metrics_folder, f"{metric}_{phase}_ha.csv")
    arr = pd.read_csv(file, header=None, index_col=[0]).to_numpy()
    avg_bpm = np.nanmean(arr[1:, 1:], axis=1)
    ha.append(avg_bpm)

    file = os.path.join(metrics_folder, f"{metric}_{phase}_la.csv")
    arr = pd.read_csv(file, header=None, index_col=[0]).to_numpy()
    avg_bpm = np.nanmean(arr[1:, 1:], axis=1)
    la.append(avg_bpm)

ha_arr = np.asarray(ha).transpose()
la_arr = np.asarray(la).transpose()

x = np.vstack([ha_arr, la_arr])
x = np.nan_to_num(x)
y = np.asarray([1 for _ in range(ha_arr.shape[0])] + [0 for _ in range(ha_arr.shape[0])])


clf = svm.SVC()
for i in [1, 12, 15, 16, 19, 22, 40, 41]:
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=i)

    clf.fit(x_train, y_train)

    preds = clf.predict(x_test)
    actual = y_test

    acc = accuracy_score(actual, preds)
    print(f"Acc: {acc}")

C:\Users\zhoux\AppData\Local\Temp/ipykernel_27024/4106669846.py:28: RuntimeWarning: Mean of empty slice
  avg_bpm = np.nanmean(arr[1:, 1:], axis=1)


Acc: 0.8333333333333334
Acc: 0.3333333333333333
Acc: 0.16666666666666666
Acc: 0.5
Acc: 0.8333333333333334
Acc: 0.16666666666666666
Acc: 0.3333333333333333
Acc: 0.6666666666666666


In [70]:
# SVM -- ECG LF RR
importlib.reload(preprocessing)
importlib.reload(dr)
importlib.reload(dt)

import csv

metrics_folder = os.path.join(dr.Paths.DATA_DIR, "metrics")
metric = "lf_rr"
phases = [
    dr.Phases.BASE_REST, dr.Phases.BASE_SPEECH,
    dr.Phases.BUG_RELAX, dr.Phases.BUG_ANTICIPATE, dr.Phases.BUG_EXPOSURE, dr.Phases.BUG_BREAK, dr.Phases.BUG_REFLECT,
    dr.Phases.SPEECH_RELAX, dr.Phases.SPEECH_ANTICIPATE, dr.Phases.SPEECH_EXPOSURE, dr.Phases.SPEECH_BREAK, dr.Phases.SPEECH_REFLECT

]

ha = []
la = []

for phase in phases:
    file = os.path.join(metrics_folder, f"{metric}_{phase}_ha.csv")
    arr = pd.read_csv(file, header=None, index_col=[0]).to_numpy()
    avg_bpm = np.nanmean(arr[1:, 1:], axis=1)
    ha.append(avg_bpm)

    file = os.path.join(metrics_folder, f"{metric}_{phase}_la.csv")
    arr = pd.read_csv(file, header=None, index_col=[0]).to_numpy()
    avg_bpm = np.nanmean(arr[1:, 1:], axis=1)
    la.append(avg_bpm)

ha_arr = np.asarray(ha).transpose()
la_arr = np.asarray(la).transpose()

x = np.vstack([ha_arr, la_arr])
x = np.nan_to_num(x)
y = np.asarray([1 for _ in range(ha_arr.shape[0])] + [0 for _ in range(ha_arr.shape[0])])


clf = svm.SVC()
for i in [1, 12, 15, 16, 19, 22, 40, 41]:
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=i)

    clf.fit(x_train, y_train)

    preds = clf.predict(x_test)
    actual = y_test

    acc = accuracy_score(actual, preds)
    print(f"Acc: {acc}")

C:\Users\zhoux\AppData\Local\Temp/ipykernel_27024/1027855165.py:28: RuntimeWarning: Mean of empty slice
  avg_bpm = np.nanmean(arr[1:, 1:], axis=1)


Acc: 0.6666666666666666
Acc: 0.3333333333333333
Acc: 0.16666666666666666
Acc: 0.5
Acc: 0.8333333333333334
Acc: 0.16666666666666666
Acc: 0.3333333333333333
Acc: 0.5


In [71]:
# SVM -- EDA MEAN SCL
importlib.reload(preprocessing)
importlib.reload(dr)
importlib.reload(dt)

import csv

metrics_folder = os.path.join(dr.Paths.DATA_DIR, "metrics")
metric = "mean_SCL"
phases = [
    dr.Phases.BASE_REST, dr.Phases.BASE_SPEECH,
    dr.Phases.BUG_RELAX, dr.Phases.BUG_ANTICIPATE, dr.Phases.BUG_EXPOSURE, dr.Phases.BUG_BREAK, dr.Phases.BUG_REFLECT,
    dr.Phases.SPEECH_RELAX, dr.Phases.SPEECH_ANTICIPATE, dr.Phases.SPEECH_EXPOSURE, dr.Phases.SPEECH_BREAK, dr.Phases.SPEECH_REFLECT

]

ha = []
la = []

for phase in phases:
    file = os.path.join(metrics_folder, f"{metric}_{phase}_ha.csv")
    arr = pd.read_csv(file, header=None, index_col=[0]).to_numpy()
    avg_bpm = np.nanmean(arr[1:, 1:], axis=1)
    ha.append(avg_bpm)

    file = os.path.join(metrics_folder, f"{metric}_{phase}_la.csv")
    arr = pd.read_csv(file, header=None, index_col=[0]).to_numpy()
    avg_bpm = np.nanmean(arr[1:, 1:], axis=1)
    la.append(avg_bpm)

ha_arr = np.asarray(ha).transpose()
la_arr = np.asarray(la).transpose()

x = np.vstack([ha_arr, la_arr])
x = np.nan_to_num(x)
y = np.asarray([1 for _ in range(ha_arr.shape[0])] + [0 for _ in range(ha_arr.shape[0])])


clf = svm.SVC()
for i in [1, 12, 15, 16, 19, 22, 40, 41]:
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=i)

    clf.fit(x_train, y_train)

    preds = clf.predict(x_test)
    actual = y_test

    acc = accuracy_score(actual, preds)
    print(f"Acc: {acc}")

Acc: 0.3333333333333333
Acc: 0.16666666666666666
Acc: 0.6666666666666666
Acc: 0.5
Acc: 0.8333333333333334
Acc: 0.3333333333333333
Acc: 0.3333333333333333
Acc: 0.5


In [72]:
# SVM -- EDA SCR RATE
importlib.reload(preprocessing)
importlib.reload(dr)
importlib.reload(dt)

import csv

metrics_folder = os.path.join(dr.Paths.DATA_DIR, "metrics")
metric = "SCR_rate"
phases = [
    dr.Phases.BASE_REST, dr.Phases.BASE_SPEECH,
    dr.Phases.BUG_RELAX, dr.Phases.BUG_ANTICIPATE, dr.Phases.BUG_EXPOSURE, dr.Phases.BUG_BREAK, dr.Phases.BUG_REFLECT,
    dr.Phases.SPEECH_RELAX, dr.Phases.SPEECH_ANTICIPATE, dr.Phases.SPEECH_EXPOSURE, dr.Phases.SPEECH_BREAK, dr.Phases.SPEECH_REFLECT

]

ha = []
la = []

for phase in phases:
    file = os.path.join(metrics_folder, f"{metric}_{phase}_ha.csv")
    arr = pd.read_csv(file, header=None, index_col=[0]).to_numpy()
    avg_bpm = np.nanmean(arr[1:, 1:], axis=1)
    ha.append(avg_bpm)

    file = os.path.join(metrics_folder, f"{metric}_{phase}_la.csv")
    arr = pd.read_csv(file, header=None, index_col=[0]).to_numpy()
    avg_bpm = np.nanmean(arr[1:, 1:], axis=1)
    la.append(avg_bpm)

ha_arr = np.asarray(ha).transpose()
la_arr = np.asarray(la).transpose()

x = np.vstack([ha_arr, la_arr])
x = np.nan_to_num(x)
y = np.asarray([1 for _ in range(ha_arr.shape[0])] + [0 for _ in range(ha_arr.shape[0])])


clf = svm.SVC()
for i in [1, 12, 15, 16, 19, 22, 40, 41]:
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=i)

    clf.fit(x_train, y_train)

    preds = clf.predict(x_test)
    actual = y_test

    acc = accuracy_score(actual, preds)
    print(f"Acc: {acc}")

Acc: 0.5
Acc: 0.16666666666666666
Acc: 0.5
Acc: 0.8333333333333334
Acc: 0.6666666666666666
Acc: 0.3333333333333333
Acc: 0.16666666666666666
Acc: 0.5
